<a href="https://colab.research.google.com/github/shubhusararthy/formi/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.8 MB/s eta 0:00:00


In [13]:
# Install necessary libraries in Colab
!pip install python-docx PyPDF2

import os
from pathlib import Path
from docx import Document
from PyPDF2 import PdfReader
import re

# --- Folder finding logic (as before) ---
possible_locations = [
    Path('/content/knowledge base'),
    Path('/content/knowledge_base'),
    Path('knowledge base'),
    Path('./knowledge base'),
    Path('/content/Knowledge Base'),
    Path.home() / 'knowledge base',
]

folder_found = None
for folder_path in possible_locations:
    if folder_path.exists() and folder_path.is_dir():
        folder_found = folder_path.resolve()
        break

if folder_found is None:
    raise FileNotFoundError("'knowledge base' folder not found")

# --- Read files and extract text ---
all_texts = []
for file_path in folder_found.glob('*'):
    if not file_path.is_file():
        continue
    try:
        if file_path.suffix.lower() == '.docx':
            doc = Document(file_path)
            text = '\n'.join(p.text for p in doc.paragraphs if p.text.strip())
        elif file_path.suffix.lower() == '.pdf':
            with open(file_path, 'rb') as f:
                reader = PdfReader(f)
                text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
        elif file_path.suffix.lower() in ['.txt']:
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
        else:
            continue
        if text.strip():
            all_texts.append({'file_name': file_path.name, 'content': text})
    except Exception as e:
        print(f"Error processing {file_path.name}: {str(e)}")

# --- Chunking function ---
def chunk_text(text, max_tokens=800):
    max_chars = max_tokens * 4
    chunks = []
    paragraphs = [p for p in text.split('\n') if p.strip()]
    current_chunk = []
    current_length = 0
    for para in paragraphs:
        para_length = len(para)
        if current_length + para_length > max_chars:
            if current_chunk:
                chunks.append('\n'.join(current_chunk))
                current_chunk = []
                current_length = 0
        current_chunk.append(para)
        current_length += para_length
    if current_chunk:
        chunks.append('\n'.join(current_chunk))
    if not chunks:
        chunks = [text[i:i+max_chars] for i in range(0, len(text), max_chars)]
    return [c for c in chunks if c.strip()]

# --- Metadata extraction from file name ---
def extract_metadata(file_name):
    # Example: Bangalore_Indiranagar_general_1.docx
    pattern = r'(?P<city>[^_]+)_(?P<location>[^_]+)_(?P<category>[^_]+)'
    match = re.match(pattern, file_name)
    if match:
        return match.group('city'), match.group('location'), match.group('category')
    else:
        return 'Unknown', 'Unknown', 'Unknown'

# --- Build chunks with metadata ---
all_chunks = []
for file in all_texts:
    city, location, category = extract_metadata(file['file_name'])
    chunks = chunk_text(file['content'])
    for idx, chunk in enumerate(chunks):
        all_chunks.append({
            'city': city.replace('_', ' '),
            'location': location.replace('_', ' '),
            'category': category,
            'chunk': chunk
        })

# --- Output ---
print(f"Total number of chunks created: {len(all_chunks)}")
if all_chunks:
    print("Sample chunk with metadata:")
    print(all_chunks[0])
else:
    print("No chunks were created.")


Total number of chunks created: 28
Sample chunk with metadata:
{'city': 'Bangalore ', 'location': ' Electronic City ', 'category': ' Barbeque Nation.docx', 'chunk': "5/12/25, 2:20 PM Bangalore : Electronic City : Barbeque Nation \nBangalore : Electronic City \nModified on: Sun, 8 Sep 2024 1:50 PM \nBaby Chair \nhttps://barbequenation.freshservice.com/support/solutions/articles/27000026308-Bangalore-Electronic-City 1/4\n5/12/25, 2:20 PM Bangalore : Electronic City : Barbeque Nation \nYou have been inactive for a long time. For security \nreasons, you will be automatically logged out in 0 \nYes \nPDR Minimum Pax Required. No \n7090757126 \nseconds. Click anywhere on the screen to stay logged in. \nhttps://barbequenation.freshservice.com/support/solutions/articles/27000026308-Bangalore-Electronic-City 2/4\n5/12/25, 2:20 PM Bangalore : Electronic City : Barbeque Nation \nYour session is about to expire! \nStudent Offe No \nDrink's Offe \nhttps://barbequenation.freshservice.com/support/solu

In [14]:
def find_relevant_chunk(city, location, query, chunks):
    # Simple keyword search in relevant city/location
    candidates = [
        c for c in chunks
        if city.lower() in c['city'].lower() and location.lower() in c['location'].lower()
    ]
    # If none found, fallback to city only
    if not candidates:
        candidates = [c for c in chunks if city.lower() in c['city'].lower()]
    # Search for query keywords
    for c in candidates:
        if any(word in c['chunk'].lower() for word in query.lower().split()):
            return c['chunk']
    # Fallback: return first candidate
    return candidates[0]['chunk'] if candidates else "Sorry, I couldn't find relevant information."

def chatbot():
    print("👋 Welcome to Barbeque Nation Booking Assistant!")
    city = input("Which city are you interested in? ").strip()
    location = input("Which location/branch? ").strip()

    print(f"Great! You chose {location}, {city}.")
    print("How can I help you today? (e.g., 'I want to book a table for 4 at 7pm', 'What are your timings?', 'Is valet parking available?')")

    while True:
        user_query = input("\nYou: ").strip()
        if user_query.lower() in ['exit', 'quit', 'bye']:
            print("Thank you for chatting! Have a great day. 😊")
            break
        answer = find_relevant_chunk(city, location, user_query, all_chunks)
        print(f"🤖: {answer}")

# Run the chatbot
chatbot()


👋 Welcome to Barbeque Nation Booking Assistant!
Which city are you interested in? bangalore
Which location/branch? kormangla
Great! You chose kormangla, bangalore.
How can I help you today? (e.g., 'I want to book a table for 4 at 7pm', 'What are your timings?', 'Is valet parking available?')

You: timings
🤖: Bangalore : Electronic City
Modiﬁed on: Sun, 8 Sep 2024 1:50 PM
Monday Instruction
Pax Lunch Slot-1   Lunch Slot-2   Dinner  Slot-1   Dinner  Slot-2
2+1  Flexible    Take till last entry .    Flexible    Take till last entry .
3 & Above  Flexible    Take till last entry .    Flexible    Take till last entry .
Tuesday Instruction
Pax Lunch Slot-1   Lunch Slot-2   Dinner  Slot-1   Dinner  Slot-2
2+1  Flexible    Take till last entry .    Flexible    Take till last entry .
3 & Above  Flexible    Take till last entry .    Flexible    Take till last entry .
Wednesday Instruction
Pax Lunch Slot-1   Lunch Slot-2   Dinner  Slot-1   Dinner  Slot-2
2+1  Flexible    Take till last entry .    